In [ ]:
import osmnx as ox
import pandas as pd
import pvlib 
from datetime import datetime, date, time, timedelta 

In [ ]:
superf_techo = {'surface_tilt': 0} 
superf_derecha = {'surface_tilt': 90} 
superf_izquierda = {'surface_tilt': 90} 
superf_capo = {'surface_tilt': 15} 
superf_maletero = {'surface_tilt': 45} 
coord_origen = (40.405536, -3.700684 ) #ETSIDI
coord_destino = (40.462644, -3.763584 ) #CNI
timezone = 'Etc/GMT+2'
altitude=667
hora_salida = datetime(2020, 7, 28, 14, 30, 0) #y, m, d, h, min, s 

In [ ]:
superficies_coche =[]
superficies_coche.append(superf_techo)
superficies_coche.append(superf_derecha)
superficies_coche.append(superf_izquierda)
superficies_coche.append(superf_capo)
superficies_coche.append(superf_maletero)
len(superficies_coche)

In [ ]:
mapa_nodos = ox.graph_from_point (coord_origen, dist=7000, dist_type ='bbox' , network_type = 'drive')
fig, ax = ox.plot_graph(mapa_nodos) 
ox.speed.add_edge_speeds(mapa_nodos , hwy_speeds = 30 , fallback = 30 , precision = 0 )
ox.speed.add_edge_travel_times( mapa_nodos , precision = 0 )

In [ ]:
# #GRAFICAR A GEODATAFRAME
nodos, caminos = ox.graph_to_gdfs(mapa_nodos) #Función de OSMnx que convierte el multidigraph en geodataframes de NODOS y LÍNEAS
# pd.set_option("max_columns", None) #Showing only two columns
# pd.set_option("max_rows", None)

In [ ]:
#------------------SECCIÓN INFORMATIVA-----------------------------#
nodos.head() #GeoDataFrames de los nodos
caminos.head()

In [ ]:
#---------------------NODOS ORIGEN Y DESTINO SEGÚN COORDENADAS----------------------------------#
nodo_origen = ox.get_nearest_node(mapa_nodos, coord_origen, method='haversine', return_dist=False)
nodo_destino = ox.get_nearest_node(mapa_nodos, coord_destino, method='haversine', return_dist=False)
nodo_origen,nodo_destino

In [ ]:
#--------------------------NODOS DE LA RUTA---------------------------------------#
ruta = ox.shortest_path(G=mapa_nodos, orig=nodo_origen, dest=nodo_destino, weight='length')
print(ruta) #Obtenemos la lista de nodos que se encuentran en el camino más corto.
len(ruta)

In [ ]:
#--------------------------INFORMACIÓN CALLES DE LA RUTA ---------------------------------------#
lista_caminos =[]
for x in range(1,len(ruta)):
    lista_caminos.append(mapa_nodos[ruta[x-1]][ruta[x]])
len(lista_caminos)

In [ ]:
# #--------------------------NOMBRE DE LAS CALLES---------------------------------------#
# nombre_calles =[]
# for x in range(0,len(lista_caminos)):
#     nombre_calles.append(lista_caminos[x][0]['name'])
# len(nombre_calles)

In [ ]:
#--------------------------NODOS INICIALES DE LAS CALLES---------------------------------------#
nodo1_calle =[]
for x in range(0,len(ruta)-1):
    nodo1_calle.append(ruta[x])
len(nodo1_calle)

In [ ]:
#--------------------------NODOS FINALES DE LAS CALLES---------------------------------------#
nodo2_calle =[]
for x in range(1,len(ruta)):
    nodo2_calle.append(ruta[x])
len(nodo2_calle)

In [ ]:
#--------------------------COORDENADAS TOTALES EN UNA LISTA---------------------------------------#
lista_coords_nodos =[]
for x in range(0,len(ruta)):
    lista_coords_nodos.append(mapa_nodos.nodes[ruta[x]]['y'])
    lista_coords_nodos.append(mapa_nodos.nodes[ruta[x]]['x'])
len(lista_coords_nodos)

In [ ]:
#--------------------------COORDENADAS FINALES DE LA RUTA---------------------------------------#
coords_nodos = []
for i in range(0, len(lista_coords_nodos), 2):
    coords_nodos.append(lista_coords_nodos[i:i+2])
len(coords_nodos)

In [ ]:
#--------------------------COORDENADAS INICIALES DE LAS CALLES---------------------------------------#
coords1_calle =[]
for x in range(0,len(ruta)-1): #Elimino la ultima coordenada para tener longitud 30
    coords1_calle.append(coords_nodos[x])
len(coords1_calle) 

In [ ]:
#-------------------------COORDENADAS FINALES DE LAS CALLES--------------------------------------#
coords2_calle =[]
for x in range(1,len(ruta)):
    coords2_calle.append(coords_nodos[x])
len(coords2_calle)

In [ ]:
#----------------------------TRAZADO DE LA RUTA-------------------------------------------#
caminos.plot(ax=ax, linewidth=1, edgecolor='dimgray') #Líneas
fig2, ax = ox.plot_graph_route(mapa_nodos, ruta)

In [ ]:
#-------------------------INFORMACIÓN POR NODO DE LA RUTA----------------------------------------#
ruta_info=ox.utils_graph.get_route_edge_attributes(mapa_nodos, ruta, attribute=None, minimize_key='length', retrieve_default=None)
ruta_info

In [ ]:
#----------------------------APROXIMACIÓN DE LOS SEGUNDOS A ENTEROS-------------------------------------------#
time_aprox=[]
for x in range(0,len(ruta_info)):
    time_aprox.append(round(ruta_info[x]['travel_time']))

In [ ]:
#----------------------------VELOCIDAD POR CALLE EN KPH-------------------------------------------#
calles_speed =[]
for x in range(0,len(ruta_info)):
    calles_speed.append(ruta_info[x]['speed_kph'])
len(calles_speed)

In [ ]:
#---------HORA DE PASADA POR NODO-----------#
horas_intermedias = hora_salida
hora_llegada=hora_salida
horas_trayecto = []
for x in range(0,len(ruta_info)):
   horas_trayecto.append(horas_intermedias + timedelta(seconds=time_aprox[x]))
   horas_intermedias += timedelta(seconds=time_aprox[x])

for x in range(0,len(ruta_info)):
    hora_llegada+=timedelta(seconds=time_aprox[x])
len(horas_trayecto)

In [ ]:
coords_list=[]
for x in range(0,len(coords_nodos)):
    coords_list.append(tuple(coords_nodos[x]))
len(coords_list)

In [ ]:
#-------------FECHAS ADAPTADAS PARA EL ANÁLISIS-----------#
datetimes=[]
for x in range(1,len(horas_trayecto)):
    datetimes.append(pd.date_range(start=horas_trayecto[x-1], end=horas_trayecto[x]- timedelta(seconds=1), freq='1S'))
len(datetimes)

In [ ]:
solar_pos=[]
dni_extra=[]
airmass_rel =[]
pressure = pvlib.atmosphere.alt2pres(altitude)
airmass_abs = []
turbidez=[]
val_sol=[]
for x in range(1,len(horas_trayecto)):
    naive_times = pd.date_range(start=horas_trayecto[x-1], end=horas_trayecto[x]- timedelta(seconds=1), freq='1S')
    times = naive_times.tz_localize(timezone)
    solar_pos.append(pvlib.solarposition.get_solarposition(times, coords_list[x-1][0], coords_list[x-1][1]))
    dni_extra.append(pvlib.irradiance.get_extra_radiation(times))
    turbidez.append(pvlib.clearsky.lookup_linke_turbidity(times, coords_list[x-1][0], coords_list[x-1][1]))
    
for x in range(1,len(horas_trayecto)):
    airmass_rel.append(pvlib.atmosphere.get_relative_airmass(solar_pos[x-1]['apparent_zenith']))

for x in range(1,len(horas_trayecto)):
    airmass_abs.append(pvlib.atmosphere.get_absolute_airmass(airmass_rel[x-1], pressure))

for x in range(1,len(horas_trayecto)):
    val_sol.append(pvlib.clearsky.ineichen(solar_pos[x-1]['apparent_zenith'], airmass_abs[x-1], turbidez[x-1],
                                 dni_extra=dni_extra[x-1], altitude=altitude))

len(val_sol)

In [ ]:
#-------ACIMUT SEGÚN LOS GIROS DEL COCHE------#
import utm
import numpy as np
import math
p0 = [0, 0]
p1 = (0, 1)
v0 = np.array(p1) - np.array(p0) #FIJA

utm_resta=[]
coords_to_utm=[]
utm_list=[]
utm_tuple=[]
coords_acimut=[]
acimut=[]

for x in range(1,len(coords_list)):
    coords_to_utm.append(utm.from_latlon(coords_list[x-1][0],coords_list[x-1][1]))
    
for x in range(1,len(coords_list)):
    utm_list.append(coords_to_utm[x-1][0])
    utm_list.append(coords_to_utm[x-1][1])

for x in range(0, len(utm_list), 2):
    utm_tuple.append(utm_list[x:x+2])

for x in range(1,len(coords_list)-1):
    utm_resta.append(np.array(utm_tuple[x]) - np.array(utm_tuple[x-1]))
    
for x in range(1,len(coords_list)-1):
    coords_acimut.append(round(math.degrees(np.math.atan2(np.linalg.det([utm_resta[x-1],v0]),np.dot(utm_resta[x-1],v0)))))

In [ ]:
#------------ TECHO----------------------#
irrad_techo=[]
for x in range (1,len(horas_trayecto)):
    irrad_techo.append(pvlib.irradiance.get_total_irradiance(superf_techo['surface_tilt'], coords_acimut[x-1], 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_techo)
poa_global_filtro_techo=[] #Filtramos la parte de poa_global
df_global_techo=[]
poa_global_lista_techo=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_techo.append(irrad_techo[x-1]['poa_global'])
    df_global_techo.append(pd.DataFrame(poa_global_filtro_techo[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_techo.append(df_global_techo[x-1].iloc[:, 0]) # Primera columna
    
poa_global_techo=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_techo.append(poa_global_lista_techo[x-1])
poa_global_techo[1][0]
len(poa_global_techo)


In [ ]:
#------------ PARED DERECHA----------------------#
irrad_derecha=[]
for x in range (1,len(horas_trayecto)):
    irrad_derecha.append(pvlib.irradiance.get_total_irradiance(superf_derecha['surface_tilt'], coords_acimut[x-1]+90, 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_derecha)
poa_global_filtro_derecha=[] #Filtramos la parte de poa_global
df_global_derecha=[]
poa_global_lista_derecha=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_derecha.append(irrad_derecha[x-1]['poa_global'])
    df_global_derecha.append(pd.DataFrame(poa_global_filtro_derecha[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_derecha.append(df_global_derecha[x-1].iloc[:, 0]) # Primera columna
    
poa_global_derecha=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_derecha.append(poa_global_lista_derecha[x-1])
poa_global_derecha[1][0]
len(poa_global_derecha)

In [ ]:
#------------ PARED IZQUIERDA----------------------#
irrad_izquierda=[]
for x in range (1,len(horas_trayecto)):
    irrad_izquierda.append(pvlib.irradiance.get_total_irradiance(superf_izquierda['surface_tilt'], coords_acimut[x-1]-90, 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_izquierda)
poa_global_filtro_izquierda=[] #Filtramos la parte de poa_global
df_global_izquierda=[]
poa_global_lista_izquierda=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_izquierda.append(irrad_izquierda[x-1]['poa_global'])
    df_global_izquierda.append(pd.DataFrame(poa_global_filtro_izquierda[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_izquierda.append(df_global_izquierda[x-1].iloc[:, 0]) # Primera columna
    
poa_global_izquierda=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_izquierda.append(poa_global_lista_izquierda[x-1])
poa_global_izquierda[1][0]
len(poa_global_izquierda)

In [ ]:
#------------ CAPÓ----------------------#
irrad_capo=[]
for x in range (1,len(horas_trayecto)):
    irrad_capo.append(pvlib.irradiance.get_total_irradiance(superf_capo['surface_tilt'], coords_acimut[x-1], 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_capo)
poa_global_filtro_capo=[] #Filtramos la parte de poa_global
df_global_capo=[]
poa_global_lista_capo=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_capo.append(irrad_capo[x-1]['poa_global'])
    df_global_capo.append(pd.DataFrame(poa_global_filtro_capo[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_capo.append(df_global_capo[x-1].iloc[:, 0]) # Primera columna
    
poa_global_capo=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_capo.append(poa_global_lista_capo[x-1])
poa_global_capo[1][0]
len(poa_global_capo)

In [ ]:
#------------ MALETERO----------------------#
irrad_maletero=[]
for x in range (1,len(horas_trayecto)):
    irrad_maletero.append(pvlib.irradiance.get_total_irradiance(superf_maletero['surface_tilt'], coords_acimut[x-1]+180, 
    solar_pos[x-1]['zenith'] , solar_pos[x-1]['azimuth'], val_sol[x-1]['dni'], val_sol[x-1]['ghi'] , 
    val_sol[x-1]['dhi'], dni_extra=dni_extra[x-1], airmass=airmass_abs[x-1], albedo=0.10,
    surface_type=None, model='isotropic', model_perez='allsitescomposite1990'))
len(irrad_maletero)
poa_global_filtro_maletero=[] #Filtramos la parte de poa_global
df_global_maletero=[]
poa_global_lista_maletero=[]
for x in range(1,len(horas_trayecto)):
    poa_global_filtro_maletero.append(irrad_maletero[x-1]['poa_global'])
    df_global_maletero.append(pd.DataFrame(poa_global_filtro_maletero[x-1]))
    
for x in range(1,len(horas_trayecto)):   
    poa_global_lista_maletero.append(df_global_maletero[x-1].iloc[:, 0]) # Primera columna
    
poa_global_maletero=[] 
for x in range(1,len(horas_trayecto)): 
    poa_global_maletero.append(poa_global_lista_maletero[x-1])
poa_global_maletero[1][0]
len(poa_global_maletero)

In [ ]:
#---------EXTENSIÓN DEL DATAFRAME-------#
# nombre_calles_pd=[]
calles_speed_pd=[]
time_aprox_pd=[]
nodo1_calle_pd=[]
coords1_calle_pd=[]
nodo2_calle_pd=[]
coords2_calle_pd=[]
horas_trayecto_pd =[]
poa_global_techo_pd=[]
poa_global_derecha_pd=[]
poa_global_izquierda_pd=[]
poa_global_capo_pd=[]
poa_global_maletero_pd=[]
angulo_pd=[]
for x in range(1,len(horas_trayecto)):
    for i in range(0,len(poa_global_techo[x-1])): 
#         nombre_calles_pd.append(nombre_calles[x-1])
        calles_speed_pd.append(calles_speed[x-1])
        time_aprox_pd.append(time_aprox[x])
        nodo1_calle_pd.append(nodo1_calle[x-1])
        coords1_calle_pd.append(coords_list[x-1])
        nodo2_calle_pd.append(nodo2_calle[x-1])
        coords2_calle_pd.append(coords_list[x])    
        horas_trayecto_pd.append(datetimes[x-1][i])
        poa_global_techo_pd.append(poa_global_techo[x-1][i])
        poa_global_derecha_pd.append(poa_global_derecha[x-1][i])
        poa_global_izquierda_pd.append(poa_global_izquierda[x-1][i])
        poa_global_capo_pd.append(poa_global_capo[x-1][i])
        poa_global_maletero_pd.append(poa_global_maletero[x-1][i])
        angulo_pd.append(coords_acimut[x-1])

In [ ]:
#----------------------------------------DATAFRAME-------------------------------------------#
df = pd.DataFrame(nodo1_calle_pd,columns=['Nodo Origen'])
df['Origen'] = coords1_calle_pd
df['Nodo Destino'] = nodo2_calle_pd
df['Destino'] = coords2_calle_pd
df['km/h'] = calles_speed_pd
df['Tiempo'] = time_aprox_pd
df['Ruta'] = horas_trayecto_pd
df['Azimuth']=angulo_pd
df['Techo'] = poa_global_techo_pd
df['Derecha'] = poa_global_derecha_pd
df['Izquierda'] = poa_global_izquierda_pd
df['Capó'] = poa_global_capo_pd
df['Maletero'] = poa_global_maletero_pd

In [ ]:
pd.set_option("max_columns", None) #Showing only two columns
pd.set_option("max_rows", None)
df